In [18]:
# General Libraries
import numpy as np
from tdmclient import ClientAsync
import time
import matplotlib.pyplot as plt

# Custom libraries 
import filtering
import local_navigation as ln

client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node ad6b0acc-2bf0-4a27-bb7a-64d9d8bda21d

In [29]:
#MAIN CODE : 

THRESHOLD_OBST = 3500 
command_motor = [100, 100]
thymio_data = []
speed_history = []
angular_speed_history = []
camera_position_histoty = []
norm_distance = []
angle = []

async def main():

    local_obstacle = False
    start_time = time.time()
    node = await client.wait_for_node()
    state_estimation = np.array([[0], [0], [0], [0], [0]])
    P_estimation = 1000 * np.ones(5)

    while(1):
        sens = await ln.get_proximity_values(node)
        if (sum(sens[i] > THRESHOLD_OBST for i in range(0, 5)) > 0): local_obstacle = True

        if(local_obstacle): break

        if not local_obstacle: 
            node.send_set_variables(ln.motors(command_motor[0], command_motor[1]))
            state_estimation, P_estimation, speed, angular_speed, start_time = await filtering.get_position(client, node, thymio_data, state_estimation, P_estimation, start_time)
            speed_history.append(speed)
            angular_speed_history.append(angular_speed)
            print("state_estimate angle", state_estimation[-1][2]*180/np.pi)
            norm_distance.append(np.sqrt(state_estimation[-1][0]**2 + state_estimation[-1][1]**2))
            print("distance", norm_distance[-1])
            angle.append((state_estimation[-1][3]*180)/np.pi)
            print("angle", angle[-1])

await main()

TypeError: get_position() takes 4 positional arguments but 6 were given

In [30]:
await node.set_variables(ln.motors(0, 0))

In [ ]:
std_speed = np.std(speed_history[30:])
std_angular_speed = np.std(angular_speed_history[30:])
print("The speed variance in mm^2/s^2 is {}".format(std_speed))
print("The angular speed variance in rad^2/s^2 is {}".format(std_angular_speed))
print(len(speed_history))

plt.plot(norm_distance[:], label="distance")
plt.xlabel("Time step")
plt.ylabel("distance (mm)")
plt.legend()

plt.plot(angle[:], label="angle")
plt.xlabel("Time step")
plt.ylabel("angle (deg)")
plt.legend()